Cleans data for voter records that are formatted a certain way

In [1]:
import pandas as pd
import numpy as np
import os
from pathfinder import csvfinder

csv_file = r"2021-Mayor-Cast-Vote-Record.csv" #Change file name here/other data cleaning tasks below when dealing with different datasets
file_path = csvfinder(csv_file)

#Load the CSV file
data = pd.read_csv(file_path)

In [2]:
data = pd.read_csv(file_path)
data.rename(columns={
    '1st Choice': '1st',
    '2nd Choice': '2nd',
    '3rd Choice': '3rd',
    'Precinct': 'Precinct',
    'Count': 'Count'
}, inplace=True)
data.to_csv(file_path, index = False)

Remove invalid votes

In [3]:
#Define the list of values to filter out
undesired_values = ["undervote", "overvote", "defective"]

cleaned_data = data.copy()

#Replace undesired values with NA
cleaned_data.replace(undesired_values, np.nan, inplace=True)

# Filter out rows where all of the specified columns have undesired values
# cleaned_data = data[~data[['1st', '2nd', '3rd']].isin(undesired_values).any(axis=1)]
#cleaned_data = data[~(data['1st'].isin(undesired_values) &
 #                     data['2nd'].isin(undesired_values) &
 #                     data['3rd'].isin(undesired_values))]
cleaned_data = cleaned_data.dropna(subset=['1st', '2nd', '3rd'], how='all')

#Drop Precinct column
cleaned_data = cleaned_data.drop(columns=['Precinct'])

#Display the first few rows of the cleaned dataframe to verify the changes
cleaned_data

data.to_csv(file_path, index = False)

In [4]:
print(cleaned_data.groupby(['1st', '2nd', '3rd']).agg({'Count': 'sum'}).reset_index().sort_values(by='Count', ascending=True, ignore_index=True))

                   1st              2nd                   3rd  Count
0        Sheila Nezhad  Paul E. Johnson          Clint Conner      1
1          Doug Nelson    Sheila Nezhad          Clint Conner      1
2          Mark Globus       Kate Knuth           Mike Winter      1
3     Troy Benjegerdes    Sheila Nezhad   Christopher W David      1
4          Doug Nelson    Sheila Nezhad  Kevin "No Body" Ward      1
...                ...              ...                   ...    ...
2895        Jacob Frey       Jacob Frey            Jacob Frey   1974
2896        Kate Knuth    Sheila Nezhad            Jacob Frey   2511
2897        Jacob Frey       Kate Knuth         Sheila Nezhad   2720
2898        Kate Knuth    Sheila Nezhad               AJ Awed   2995
2899     Sheila Nezhad       Kate Knuth               AJ Awed   4330

[2900 rows x 4 columns]


Edit invalid choices in votes

In [5]:
# Identify rows where 1st == 2nd == 3rd
condition_all = (cleaned_data['1st'] == cleaned_data['2nd']) & (cleaned_data['1st'] == cleaned_data['3rd']) & (cleaned_data["1st"] != "UWI")

# Change col2 and col3 to NaN for those rows
cleaned_data.loc[condition_all, ['2nd', '3rd']] = np.nan

# Identify rows
condition_1st_3rd_equal = (cleaned_data['1st'] == cleaned_data['3rd']) & (cleaned_data["1st"] != "UWI")
condition_2nd_3rd_equal = (cleaned_data['2nd'] == cleaned_data['3rd']) & (cleaned_data["2nd"] != "UWI")
condition_1st_2nd_equal = (cleaned_data['1st'] == cleaned_data['2nd']) & (cleaned_data["1st"] != "UWI")

# Change 3rd to NaN for those rows
cleaned_data.loc[condition_1st_3rd_equal, '3rd'] = np.nan
cleaned_data.loc[condition_2nd_3rd_equal, '3rd'] = np.nan
cleaned_data.loc[condition_1st_2nd_equal, '2nd'] = np.nan

# Check if '1st' and '2nd' are NaN
condition = cleaned_data['1st'].isna() & cleaned_data['2nd'].isna()

# Apply the condition and update the values
cleaned_data.loc[condition, '1st'] = cleaned_data.loc[condition, '3rd']
cleaned_data.loc[condition, '3rd'] = np.nan

# Check if '2nd' are NaN
condition = cleaned_data['2nd'].isna()

# Apply the condition and update the values
cleaned_data.loc[condition, '2nd'] = cleaned_data.loc[condition, '3rd']
cleaned_data.loc[condition, '3rd'] = np.nan

# Check if '1st' are NaN
condition = cleaned_data['1st'].isna()

# Apply the condition and update the values
cleaned_data.loc[condition, '1st'] = cleaned_data.loc[condition, '2nd']
cleaned_data.loc[condition, '2nd'] = np.nan

cleaned_data

cleaned_data.to_csv(file_path, index = False)

Aggregate Data

In [6]:
cleaned_data = cleaned_data.fillna('Null')
aggregated_data = cleaned_data.groupby(['1st', '2nd', '3rd']).agg({'Count': 'sum'}).reset_index().sort_values(by='1st', ascending=True, ignore_index=True)
print(aggregated_data)

cleaned_data.to_csv(file_path, index = False)

          1st                    2nd                         3rd  Count
0     AJ Awed  Bob "Again" Carney Jr                  Jacob Frey      4
1     AJ Awed            Mike Winter        Kevin "No Body" Ward      2
2     AJ Awed            Mike Winter              Laverne Turner      1
3     AJ Awed            Mike Winter                 Mark Globus      3
4     AJ Awed            Mike Winter  Nate "Honey Badger" Atkins      1
...       ...                    ...                         ...    ...
2951      UWI             Jacob Frey  Nate "Honey Badger" Atkins      2
2952      UWI             Jacob Frey                        Null      4
2953      UWI             Jacob Frey               Sheila Nezhad      1
2954      UWI           Clint Conner                        Null      1
2955      UWI                    UWI                         UWI     31

[2956 rows x 4 columns]


Round 1 1st Rank

In [7]:
# Group by the first-ranked candidate and sum the counts, then convert to DataFrame
first_rank_counts_df = aggregated_data.groupby('1st')['Count'].sum().reset_index()

# Rename the columns
first_rank_counts_df.columns = ['Candidate', 'Count']

# Set the index to start from 1
first_rank_counts_df.index = first_rank_counts_df.index + 1

print(first_rank_counts_df)

cleaned_data.to_csv(file_path, index = False)

                            Candidate  Count
1                             AJ Awed   6860
2               Bob "Again" Carney Jr   2788
3   Christopher Robin "CRZ" Zimmerman      2
4                 Christopher W David    493
5                        Clint Conner   4309
6                         Doug Nelson    739
7                          Jacob Frey  61620
8                          Kate Knuth  26468
9                Kevin "No Body" Ward    282
10                     Laverne Turner   4620
11                      Marcus Harcus   1189
12                        Mark Globus   1158
13                        Mike Winter    642
14         Nate "Honey Badger" Atkins   1179
15                    Paul E. Johnson    243
16                     Perry, Jerrell    687
17                      Sheila Nezhad  30368
18                   Troy Benjegerdes    184
19                                UWI    143


Next round: remove UWI

In [8]:
cleaned_data.replace("UWI", np.nan, inplace=True)
cleaned_data.replace("Null", np.nan, inplace=True)

#Repeat Steps
cleaned_data = cleaned_data.dropna(subset=['1st', '2nd', '3rd'], how='all')

#Repeat Steps

# Check if '1st' and '2nd' are NaN
condition = cleaned_data['1st'].isna() & cleaned_data['2nd'].isna()

# Apply the condition and update the values
cleaned_data.loc[condition, '1st'] = cleaned_data.loc[condition, '3rd']
cleaned_data.loc[condition, '3rd'] = np.nan

# Check if '2nd' are NaN
condition = cleaned_data['2nd'].isna()

# Apply the condition and update the values
cleaned_data.loc[condition, '2nd'] = cleaned_data.loc[condition, '3rd']
cleaned_data.loc[condition, '3rd'] = np.nan

# Check if '1st' are NaN
condition = cleaned_data['1st'].isna()

# Apply the condition and update the values
cleaned_data.loc[condition, '1st'] = cleaned_data.loc[condition, '2nd']
cleaned_data.loc[condition, '2nd'] = np.nan

print(cleaned_data)

cleaned_data.to_csv(file_path, index = False)

                  1st                   2nd            3rd  Count
0          Jacob Frey                   NaN            NaN      1
1         Mark Globus           Mike Winter     Jacob Frey      1
2          Jacob Frey  Kevin "No Body" Ward            NaN      1
3          Jacob Frey                   NaN            NaN      1
4          Jacob Frey            Kate Knuth        AJ Awed      1
...               ...                   ...            ...    ...
145332    Doug Nelson            Kate Knuth  Sheila Nezhad      1
145333  Sheila Nezhad            Kate Knuth            NaN      1
145334     Jacob Frey               AJ Awed   Clint Conner      1
145335  Sheila Nezhad            Kate Knuth            NaN      1
145336  Sheila Nezhad               AJ Awed  Marcus Harcus      1

[143890 rows x 4 columns]


Aggregate

In [9]:
cleaned_data = cleaned_data.fillna('Null')
aggregated_data = cleaned_data.groupby(['1st', '2nd', '3rd']).agg({'Count': 'sum'}).reset_index().sort_values(by='1st', ascending=True, ignore_index=True)
print(aggregated_data)

cleaned_data.to_csv(file_path, index = False)

                   1st                    2nd                         3rd  \
0              AJ Awed  Bob "Again" Carney Jr                  Jacob Frey   
1              AJ Awed            Mike Winter        Kevin "No Body" Ward   
2              AJ Awed            Mike Winter              Laverne Turner   
3              AJ Awed            Mike Winter                 Mark Globus   
4              AJ Awed            Mike Winter  Nate "Honey Badger" Atkins   
...                ...                    ...                         ...   
2759  Troy Benjegerdes           Clint Conner               Sheila Nezhad   
2760  Troy Benjegerdes           Clint Conner                        Null   
2761  Troy Benjegerdes           Clint Conner                 Mark Globus   
2762  Troy Benjegerdes             Jacob Frey                 Doug Nelson   
2763  Troy Benjegerdes          Sheila Nezhad             Paul E. Johnson   

      Count  
0         4  
1         2  
2         1  
3         3  
4    

Round 2 first rank

In [10]:
# Group by the first-ranked candidate and sum the counts, then convert to DataFrame
first_rank_counts_df = aggregated_data.groupby('1st')['Count'].sum().reset_index()

# Rename the columns
first_rank_counts_df.columns = ['Candidate', 'Count']

# Set the index to start from 1
first_rank_counts_df.index = first_rank_counts_df.index + 1

print(first_rank_counts_df)

cleaned_data.to_csv(file_path, index = False)

                            Candidate  Count
1                             AJ Awed   6868
2               Bob "Again" Carney Jr   2792
3   Christopher Robin "CRZ" Zimmerman      2
4                 Christopher W David    493
5                        Clint Conner   4312
6                         Doug Nelson    741
7                          Jacob Frey  61639
8                          Kate Knuth  26472
9                Kevin "No Body" Ward    283
10                     Laverne Turner   4628
11                      Marcus Harcus   1194
12                        Mark Globus   1159
13                        Mike Winter    642
14         Nate "Honey Badger" Atkins   1181
15                    Paul E. Johnson    244
16                     Perry, Jerrell    687
17                      Sheila Nezhad  30369
18                   Troy Benjegerdes    184


Aggregate

In [11]:
cleaned_data = cleaned_data.fillna('Null')
aggregated_data = cleaned_data.groupby(['1st', '2nd', '3rd']).agg({'Count': 'sum'}).reset_index().sort_values(by='1st', ascending=True, ignore_index=True)
print(aggregated_data)

cleaned_data.to_csv(file_path, index = False)

                   1st                    2nd                         3rd  \
0              AJ Awed  Bob "Again" Carney Jr                  Jacob Frey   
1              AJ Awed            Mike Winter        Kevin "No Body" Ward   
2              AJ Awed            Mike Winter              Laverne Turner   
3              AJ Awed            Mike Winter                 Mark Globus   
4              AJ Awed            Mike Winter  Nate "Honey Badger" Atkins   
...                ...                    ...                         ...   
2759  Troy Benjegerdes           Clint Conner               Sheila Nezhad   
2760  Troy Benjegerdes           Clint Conner                        Null   
2761  Troy Benjegerdes           Clint Conner                 Mark Globus   
2762  Troy Benjegerdes             Jacob Frey                 Doug Nelson   
2763  Troy Benjegerdes          Sheila Nezhad             Paul E. Johnson   

      Count  
0         4  
1         2  
2         1  
3         3  
4    

Round 3 first rank

In [12]:
# Group by the first-ranked candidate and sum the counts, then convert to DataFrame
first_rank_counts_df = aggregated_data.groupby('1st')['Count'].sum().reset_index()

# Rename the columns
first_rank_counts_df.columns = ['Candidate', 'Count']

# Set the index to start from 1
first_rank_counts_df.index = first_rank_counts_df.index + 1

print(first_rank_counts_df)

                            Candidate  Count
1                             AJ Awed   6868
2               Bob "Again" Carney Jr   2792
3   Christopher Robin "CRZ" Zimmerman      2
4                 Christopher W David    493
5                        Clint Conner   4312
6                         Doug Nelson    741
7                          Jacob Frey  61639
8                          Kate Knuth  26472
9                Kevin "No Body" Ward    283
10                     Laverne Turner   4628
11                      Marcus Harcus   1194
12                        Mark Globus   1159
13                        Mike Winter    642
14         Nate "Honey Badger" Atkins   1181
15                    Paul E. Johnson    244
16                     Perry, Jerrell    687
17                      Sheila Nezhad  30369
18                   Troy Benjegerdes    184
